In [0]:
rdd1=spark.sparkContext.parallelize(range(0,6),1)
print(rdd1.collect())
print(rdd1.getNumPartitions())

[0, 1, 2, 3, 4, 5]
1


In [0]:
# Use parallelize with 6 partitions
rdd2 = spark.sparkContext.parallelize(range(0,25), 4)
print("parallelize : "+str(rdd2.getNumPartitions()))

rddFromFile = spark.sparkContext.textFile("/FileStore/tables/test.txt",10)
print("TextFile : "+str(rddFromFile.getNumPartitions()))

parallelize : 4
TextFile : 10


In [0]:
print(spark.sparkContext.defaultParallelism)
print("From local[1] : "+str(rdd1.getNumPartitions()))

8
From local[1] : 1


In [0]:
rdd1.saveAsTextFile("/FileStore/tables/partition")

In [0]:
partitioned_values = rdd2.glom().collect()
for i, partition in enumerate(partitioned_values):
    print(f"Partition {i}: {partition}")

Partition 0: [0, 1, 2, 3, 4, 5]
Partition 1: [6, 7, 8, 9, 10, 11]
Partition 2: [12, 13, 14, 15, 16, 17]
Partition 3: [18, 19, 20, 21, 22, 23, 24]


In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [0]:
from pyspark.sql import SparkSession
spark=spark.builder.appName('broadcast').getOrCreate()
states = {"NY":"New York", "CA":"California", "FL":"Florida","IN":"India"}
broadcast_states=spark.sparkContext.broadcast(states)
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL"),
    ("Pradeep","SV","India","IN")
  ]
columns=["Fname","Lname","country","CD"]
rdd=spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcast_states.value[code]
result=rdd.map(lambda x:(x[0],x[1],x[2],state_convert(x[3]))).collect()
print(result)


[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida'), ('Pradeep', 'SV', 'India', 'India')]


In [0]:
from pyspark.sql import SparkSession
spark=spark.builder.appName('broadcast').getOrCreate()
states = {"NY":"New York", "CA":"California", "FL":"Florida","IN":"India"}
broadcast_states=spark.sparkContext.broadcast(states)
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL"),
    ("Pradeep","SV","India","IN")
  ]
columns=["Fname","Lname","country","CD"]
df=spark.createDataFrame(data=data,schema=columns)

def state_convert(code):
    return broadcast_states.value[code]
result=df.rdd.map(lambda x:(x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
result.show()

+-------+--------+-------+----------+
|  Fname|   Lname|country|        CD|
+-------+--------+-------+----------+
|  James|   Smith|    USA|California|
|Michael|    Rose|    USA|  New York|
| Robert|Williams|    USA|California|
|  Maria|   Jones|    USA|   Florida|
|Pradeep|      SV|  India|     India|
+-------+--------+-------+----------+

